## Activity 1: Implement PropensityNet 

## Hello 👋🏻
###  🤖 Intructions 
The main Goal of this activity for you to understand the code implementation of PropensityNet and how it could be used to predict propensity scores. 

 1. Familiarise yourself with the code and make sure to understand the role of each layer in the neural network. 
 2. There are two tasks. The first one requries you to write some code, and the second asks you some questions from the pre-class work and the network. 


## Import the libraries

In [72]:
import pandas as pd
from scipy.stats import uniform
import numpy as np 
from sklearn.model_selection import train_test_split
import tensorflow as tf 
from tensorflow.keras.layers import Dropout
import matplotlib.pylab as plt
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn import metrics
sns.set()


## Generate data 

### Create Treatment Data

In [73]:
"""
We generate treatment data using a random sample.
X_1 and X_2 are two covarients. Y_t is the outcome.
and W_t is the treatment assignment.
"""

x_1_t = uniform.rvs(loc=5, scale=1, size=1000).reshape(-1,1)
x_2_t = uniform.rvs(loc=1, scale=1, size=1000).reshape(-1,1)
y_T = uniform.rvs(loc=8, scale=1, size=1000).reshape(-1,1)
w_t = np.ones(1000).reshape(-1,1)


In [74]:
#combine all of the columns to form a dataframe
treament_df = pd.DataFrame(np.concatenate([x_1_t,x_2_t,y_T,w_t], axis=1), columns=['X_1', 'X_2', 'Y', 'W'])


In [75]:
treament_df.head(5)

,X_1,X_2,Y,W
0,5.978022,1.172185,8.865158,1.0
1,5.323185,1.837666,8.357500,1.0
2,5.312050,1.381433,8.140995,1.0
3,5.022897,1.404643,8.289280,1.0
4,5.262753,1.652021,8.165740,1.0


### Create Control Data

In [76]:
#drop the treatment assignment column
control_data = treament_df.copy()
control_data = treament_df.drop(['W'], axis=1)

In [78]:
#add noise to the treatment data to generate control data
mu, sigma = 0, 0.5
noise = np.random.normal(mu, sigma, [1000,3]) 
control_data = control_data + noise

In [79]:
control_data["W"] = (np.zeros(1000).reshape(-1,1))

In [80]:
control_data.head(5)


,X_1,X_2,Y,W
0,5.907541,0.607590,9.375177,0.0
1,5.645699,1.385603,8.515292,0.0
2,5.030041,1.689527,7.401516,0.0
3,6.133297,1.744034,8.114018,0.0
4,5.508231,1.806400,8.905058,0.0


In [81]:
#Combine both the control and the test data
frames = [control_data, treament_df]
full_data = pd.concat(frames)

In [82]:
df = full_data.copy()

## Create Traning and Test sets

In [83]:
X = df[df.columns[0:3]]
Y = df[df.columns[3]]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [84]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, input_shape=(10,3), activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 10, 10)            40        
_________________________________________________________________
dropout_16 (Dropout)         (None, 10, 10)            0         
_________________________________________________________________
dense_21 (Dense)             (None, 10, 10)            110       
_________________________________________________________________
dropout_17 (Dropout)         (None, 10, 10)            0         
_________________________________________________________________
dense_22 (Dense)             (None, 10, 10)            110       
_________________________________________________________________
dropout_18 (Dropout)         (None, 10, 10)            0         
_________________________________________________________________
dense_23 (Dense)             (None, 10, 10)           

In [85]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train the Model

In [86]:
model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
47/47 [==============================] - 0s 953us/step - loss: 1.0511 - accuracy: 0.5120
Epoch 2/100
47/47 [==============================] - 0s 896us/step - loss: 0.7698 - accuracy: 0.5060
Epoch 3/100
47/47 [==============================] - 0s 871us/step - loss: 0.7178 - accuracy: 0.5027
Epoch 4/100
47/47 [==============================] - 0s 956us/step - loss: 0.7083 - accuracy: 0.4873
Epoch 5/100
47/47 [==============================] - 0s 1ms/step - loss: 0.7050 - accuracy: 0.5193
Epoch 6/100
47/47 [==============================] - 0s 1ms/step - loss: 0.6976 - accuracy: 0.5127
Epoch 7/100
47/47 [==============================] - 0s 1ms/step - loss: 0.6971 - accuracy: 0.5287
Epoch 8/100
47/47 [==============================] - 0s 1ms/step - loss: 0.6995 - accuracy: 0.5040
Epoch 9/100
47/47 [==============================] - 0s 996us/step - loss: 0.7048 - accuracy: 0.4933
Epoch 10/100
47/47 [==============================] - 0s 979us/step - loss: 0.7029 - accuracy: 0.48

## Evaluate the Model on a test set

In [87]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy of PropensityNet classifier on test set: %.2f' % (accuracy*100))

16/16 [==============================] - 0s 906us/step - loss: 0.5372 - accuracy: 0.7480
Accuracy of PropensityNet classifier on test set: 74.80


## Train a Logistic Regression Model 

In [88]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [89]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.58


## Tasks

#### 1. Print the first 10 propensity scores predicted by PropensityNet. 
#### 🐣 Hint: see the output layer of PropensityNet. Can you print the probability of each class?


#### 2. What are some key challenges to train PropensityNet?

#### 3. Other than matching, what are some uses of propensity scores?

#### 4.In the PropensityNet application, why is it important to use a drop-out rate?  


#### 5. What is an activation function? Explain why ReLu is used as an activation function

#### 6. What is a loss function? Explian why Sparse Categorical Crossentropy is used.

#### 7. What is a softmax function? Explain why it is used as an output layer


#### 8. While deciding between using logistic regression or PropensityNet to calculate the propensity score, what are some of the factors that would influence your choice?


In [156]:
df = pd.read_stata('nsw_dw.dta')
df = df.drop('data_id', axis = 1)
df.to_csv("Lalaonde.csv")


In [163]:
df = pd.read_csv("Lalaonde.csv")
X = df.copy()
X = X.drop('treat', axis = 1)
Y = df.treat
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [164]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, input_shape=(10,9), activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 10, 10)            100       
_________________________________________________________________
dropout_52 (Dropout)         (None, 10, 10)            0         
_________________________________________________________________
dense_66 (Dense)             (None, 10, 10)            110       
_________________________________________________________________
dropout_53 (Dropout)         (None, 10, 10)            0         
_________________________________________________________________
dense_67 (Dense)             (None, 10, 10)            110       
_________________________________________________________________
dropout_54 (Dropout)         (None, 10, 10)            0         
_________________________________________________________________
dense_68 (Dense)             (None, 10, 10)           

In [165]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [166]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
11/11 [==============================] - 0s 1ms/step - loss: 432.2184 - accuracy: 0.4985
Epoch 2/100
11/11 [==============================] - 0s 1ms/step - loss: 364.1935 - accuracy: 0.5225
Epoch 3/100
11/11 [==============================] - 0s 2ms/step - loss: 276.6648 - accuracy: 0.4745
Epoch 4/100
11/11 [==============================] - 0s 2ms/step - loss: 279.4624 - accuracy: 0.5285
Epoch 5/100
11/11 [==============================] - 0s 2ms/step - loss: 228.4270 - accuracy: 0.5285
Epoch 6/100
11/11 [==============================] - 0s 2ms/step - loss: 171.2484 - accuracy: 0.5796
Epoch 7/100
11/11 [==============================] - 0s 2ms/step - loss: 131.0571 - accuracy: 0.5345
Epoch 8/100
11/11 [==============================] - 0s 2ms/step - loss: 137.5061 - accuracy: 0.5135
Epoch 9/100
11/11 [==============================] - 0s 2ms/step - loss: 144.5585 - accuracy: 0.5195
Epoch 10/100
11/11 [==============================] - 0s 2ms/step - loss: 111.8700 - accura